## 데이타 증강

In [36]:
import os
import pandas as pd
import tensorflow
import numpy as np

In [37]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '../../data', 'raw_csv')
train_data_path = os.path.join(data_dir, 'merged_train.csv')

train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,1951,직장 내 괴롭힘 대화,팀장님 이거 언제까지 마무리 하면 될까요?\n무리하지 말고 넉넉하게 주말까지 다 작...
1,4756,일반 대화,"내일 날씨 어떻대?\n비 온다던데. 우산 챙겨가야 할 것 같아.\n에이, 야외 활동..."
2,1234,기타 괴롭힘 대화,야 쟤 좀 봐.\n 꼴에 유행하는 옷 입었네 \n 호박에 줄 긋는다고 수박되나 \n...
3,4767,일반 대화,오늘 수업 내용 이해했어?\n솔직히 좀 어려웠어. 너는?\n나도 몇 부분이 헷갈리더...
4,1511,직장 내 괴롭힘 대화,자네 내일 아침에 우리집 들려서 출근하게\n네?\n내가 차가 고장났어\n아. 그런데...


In [38]:
print('전체 샘플수 :', (len(train_data)))
train_data.info()

전체 샘플수 : 4637
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4637 entries, 0 to 4636
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           4637 non-null   int64 
 1   class         4637 non-null   object
 2   conversation  4637 non-null   object
dtypes: int64(1), object(2)
memory usage: 108.8+ KB


In [39]:
# 0, 1, 2, 3, 4
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']

#class_dict = {}
#or idx, class_name in enumerate(CLASS_NAMES):
#    class_dict[class_name] = idx

#train_data['class'] = train_data['class'].apply(lambda s: class_dict[s])

#train_data.head()

**한국어 불용어 사전**

In [40]:
# 불용어 제거
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣ㅏ-ㅣ?.!,]+", " ", sentence)
    
    sentence = sentence.strip()

    return sentence

In [41]:
import re

def preprocess_korean(sentence):
    # 한글 외 모든 문자 제거
    sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', sentence)
    # 연속된 공백 제거
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence


**동의어**

In [42]:
import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)

# 전체': ['전체', '전적', '전부', '총체', '전반']
# '인': ['인', '인간', '인물', '사람', '인류']
#print(wordnet)

In [43]:
# wordnet에서 동의어 리스트를 가져온다. 
# 동의어가 없을 수도 있다
def synonyms_from_wordnet(word):
    synonyms = []
    try:
        for list in wordnet[word] :
             synonyms.append(list)
    except KeyError:
        pass
    
    return synonyms

In [44]:
#'전체': ['전체', '전적', '전부', '총체', '전반']
synonyms = synonyms_from_wordnet( "전체" )
for word in synonyms : 
    print(word)

전체
전적
전부
총체
전반


In [45]:
# word list로 구성된 sentense를 동의어로 바꿔준다
def replace_to_synonyms(input_words, count): 
     
    # new_words
    new_words = []
    
    #
    word_list = list(set([word for word in input_words]))
    random.shuffle(word_list)
    
    replace_count = 0
    for word in word_list : 
        # 동의어 리스트를 가져온다. 
        synonyms = synonyms_from_wordnet( word )
        
        if len(synonyms) == 0 : continue
        
        synonym = random.choice(list(synonyms))
        if( word == synonym) : continue
            
        new_words = [synonym if wd == word else wd for wd in input_words]
        
        replace_count +=1
        
        if replace_count >= count : break
    
    # 
    #if len(new_words) > 0 : 
    #    sentence = ' '.join(new_words)
    #   new_words = sentence.split(" ")
    
       
    return new_words
               

In [46]:
def gen_sentence_by_swap_word(inputs_words, count):
    
    # new_words
    new_words = []
    
    #
#    word_list = list(set([word for word in input_words]))
#    random.shuffle(word_list)
    
    return new_words

In [47]:
def DataAugmentation(sentense, alpha_sr=0.2, aug_count=1):
    
    augmented_sentences = []
    
    # 한글만 처리
    sentense = preprocess_korean(sentense)
    sentense = sentense.split(' ')
    words = [word for word in sentense if word != ""]
    num_words = len(words)
       
    # 동의어로 단어 바꿔 문장 생성하기
    n_sr = max(1, int(alpha_sr*num_words))
    
    for _ in range(aug_count):
        a_words = replace_to_synonyms( words, n_sr )
          
        if( len(a_words) > 0 ):
            augmented_sentences.append(" ".join(a_words))
     
    
    
    # 생성된 문장들 shuffle
    random.shuffle(augmented_sentences)
    return augmented_sentences

In [53]:
new_class_list = []
new_conversation_list = []
new_idx_list = []

idx = len(train_data['conversation'])+1;
          
for i in range(len(train_data['conversation'])):
#for i in range(2):
    class_ = train_data['class'][i]
    
    num_aug = 3
    
    augmented_sentences = DataAugmentation(train_data['conversation'][i], aug_count=num_aug)
    
    if not augmented_sentences : # 리스트가 비어 있으면 True
        continue
        
    for j in range(min(num_aug, len(augmented_sentences))):
        #if not augmented_sentences[j] : continue;
        new_class_list.append( class_ )
        new_conversation_list.append(augmented_sentences[j])
        idx =idx+1
        new_idx_list.append(int(idx))

print(len(new_class_list))    
print(len(new_conversation_list)) 
        

9644
9644


In [54]:
train_data['conversation'][0:1]

0    팀장님 이거 언제까지 마무리 하면 될까요?\n무리하지 말고 넉넉하게 주말까지 다 작...
Name: conversation, dtype: object

In [55]:
new_conversation_list[0:3]

['내일 날씨 어떻대 비 온다던데 우산 챙겨가야 할 것 같아 에이 야외 활동 계획했는데 아쉽다 다른 날로 미루는 게 어때 그래야겠다 주말에는 맑다던데 좋아 그럼 토요일로 미루자 토요일 몇 시에 만날까 오전 정의 어때 점심도 같이 먹고 완벽해 어디서 만날까 시내 광장에서 보자 거기서 출발하기 좋을 것 같아',
 '야 쟤 좀 봐 꼴에 유행하는 의류 입었네 호박에 줄 긋는다고 수박되나 으엑 못생겼어 다 들리거든 그만해줘 그럼 들리라고 하는 소리지 내가 뭘 잘못했는데 니 존재 자체가 쟤는 못생기고 찐따고 공부도 못하고 친구도 없어서 전교 찐따래요',
 '야 쟤 좀 봐 꼴에 유행하는 피복 입었네 호박에 줄 긋는다고 수박되나 으엑 못생겼어 다 들리거든 그만해줘 그럼 들리라고 하는 소리지 내가 뭘 잘못했는데 니 존재 자체가 쟤는 못생기고 찐따고 공부도 못하고 친구도 없어서 전교 찐따래요']

In [56]:
aug_data = pd.DataFrame([new_idx_list, new_class_list, new_conversation_list],
                  index=['idx','class','conversation'],
                 columns=[x for x in range(len(new_class_list))])
aug_data  = aug_data .transpose()

In [57]:
aug_data['conversation'][0:2]

0    내일 날씨 어떻대 비 온다던데 우산 챙겨가야 할 것 같아 에이 야외 활동 계획했는데...
1    야 쟤 좀 봐 꼴에 유행하는 의류 입었네 호박에 줄 긋는다고 수박되나 으엑 못생겼어...
Name: conversation, dtype: object

In [66]:
aug_train_data = train_data.append(aug_data)

aug_train_data = aug_train_data.sample(frac=1).reset_index(drop=True)

aug_train_data['idx'] = [x for x in range(len(aug_train_data))]
aug_train_data = aug_train_data.reset_index(drop=True)

In [67]:
aug_train_data['conversation'][0:2]

0    어이 코털 왜 왜 아놔됐고 너 매점가서 곰보빵 두개에 콜라하나 사와라 춘추 돈없어 ...
1    쟤 먹는거 봐 토쏠려 저리 가줄래 꼴에 먹는거는 좋아해사지고 그래서 니가 돼지 인거...
Name: conversation, dtype: object

In [68]:
aug_train_data.head()

,idx,class,conversation
0,0,기타 괴롭힘 대화,어이 코털 왜 왜 아놔됐고 너 매점가서 곰보빵 두개에 콜라하나 사와라 춘추 돈없어 ...
1,1,기타 괴롭힘 대화,쟤 먹는거 봐 토쏠려 저리 가줄래 꼴에 먹는거는 좋아해사지고 그래서 니가 돼지 인거...
2,2,일반 대화,이 의류 색깔 진짜 예쁘네요 네 저도 이 색깔이 너무 마음에 들어요 이 색깔이 잘 ...
3,3,직장 내 괴롭힘 대화,야 새꺄 일로와바 일병 보니까 너네가 이미 노래방 청소했는데 우리가 거기 담당이더라...
4,4,기타 괴롭힘 대화,야 넌 체육 수행평가 만점이라며 야 좋겠다 나도 다리 병신이었으면 얼마나 좋았을까 ...


In [69]:
aug_train_data.tail()

,idx,class,conversation
14276,14276,일반 대화,오늘 저녁에 뭐해 특별한 계획은 없어 왜 새로 생긴 보드게임 찻집 가보지 않을래 오...
14277,14277,갈취 대화,승한아 나 버스비 충전해야하는데 돈이 없어 만원만 빌려주라 응 안돼 나 한달 용돈 ...
14278,14278,기타 괴롭힘 대화,애들아 공연 연습하자 이번주에 종국 평가 있어 빨리 하자 야 김정미 너는 왜 맨날 ...
14279,14279,일반 대화,요즘 어떤 운동 하고 있어 나는 요가를 하고 있어 몸이 유연해져 오 요가 좋지 어떤...
14280,14280,직장 내 괴롭힘 대화,장원씨 오늘 왜이렇게 조용해요?\n일하니까요.\n말 좀 해봐요 노래 부르던지 \n네...


In [70]:
print(len(aug_train_data))

14281


In [71]:
class_counts = aug_train_data['class'].value_counts()

print(class_counts)

갈취 대화          3278
기타 괴롭힘 대화      2997
협박 대화          2826
직장 내 괴롭힘 대화    2758
일반 대화          2422
Name: class, dtype: int64


In [72]:
aug_train_data.to_csv('../../data/raw_csv/aug_train.csv', index=False, encoding='utf-8-sig')  # index=False는 인덱스를 저장하지 않도록 함